<a href="https://colab.research.google.com/github/divyaagrwl/Major-Project/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix,f1_score,precision_score,recall_score
wordnet_lemmatizer = WordNetLemmatizer()

In [ ]:
df = pd.read_csv('/content/Tweets.csv')
df.head()

In [ ]:
def normalizer(tweet):
  only_letters = re.sub("[^a-zA-Z]","",tweet)
  only_letters = only_letters.lower()
  only_letters = only_letters.split()
  filtered_result = [word for word in only_letters if word not in stopwords.words('english')]
  lemmas = [wordnet_lemmatizer.lemmatize(t) for t in filtered_result]
  lemmas = ' '.join(lemmas)
  return lemmas
print(normalizer)

In [1]:
df = shuffle(df)
y = df['sentiment']
x=df.text.apply(normalizer)

NameError: ignored

In [ ]:
vectorizer = CountVectorizer()
x_vectorized = vectorizer.fit_transform(x)

In [ ]:
df = df.reindex(labels=df.columns,axis=1)

In [ ]:
train_x,val_x,train_y,val_y=train_test_split(x_vectorized,y)

In [ ]:
regressor = LogisticRegression(multi_class='multinomial',solver='newton-cg')
model = regressor.fit(train_x,train_y)

In [ ]:
params = {'C':[0.001,0.01,0.1,1,10,100,1000]}
gs_clf = GridSearchCV(model,params,n_jobs=1,cv=5)
gs_clf = gs_clf.fit(train_x,train_y)
model = gs_clf.best_estimator_

In [ ]:
y_pred = model.predict(val_x)
_f1 = f1_score(val_y,y_pred,average='micro')
_confusion= confusion_matrix(val_y,y_pred)
__precision = precision_score(val_y,y_pred,average='micro')
_recall = recall_score(val_y,y_pred,average='micro')
_statistics = {
                'f1_score':_f1,
                'confusion_matrix':_confusion,
                'precision_score':__precision,
                'recall':_recall
              }

In [ ]:
print(_statistics)

In [ ]:
test_feature = vectorizer.transform(['Positive'])
model.predict(test_feature)

In [ ]:
test_feature = vectorizer.transform(['Negative'])
model.predict(test_feature)

In [ ]:
test_feature = vectorizer.transform(['Neutral'])
model.predict(test_feature)

In [ ]:
import joblib
joblib.dump(model,'model')

In [ ]:
!pip install streamlit --quiet
!pip install pyngrok==4.1.1 --quiet
from pyngrok import ngrok

In [ ]:
%%writefile app.py
import streamlit as st
import sklearn
import joblib
model = joblib.load('model')
st.title('Sentiment Analyser')
ip = st.text_input('Enter your remarks:')
ip = ip.reshape(-1,1)
op = model.predict([ip])
if st.button('Predict'):
  st.title(op[0])

In [ ]:
!nohup streamlit run app.py &
url=ngrok.connect(port='8501')
url